In [7]:
import pandas as pd
import ast
from itertools import combinations, chain

In [1]:
import chromadb

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.5/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.5/u/import-error

In [3]:
# Import data
drivers = pd.read_csv('F1_Fantasy\\Fantasy2023\\F1_23_drivers_basic.csv')
constructors = pd.read_csv('F1_Fantasy\\Fantasy2023\\F1_23_constructors_basic.csv')

In [3]:
drivers

,abbreviation,color,constructor,race_results
0,VER,#0600ef,False,[{'fantasy_results': [{'id': 'quali_not_classi...
1,PER,#0600ef,False,[{'fantasy_results': [{'id': 'quali_not_classi...
2,HAM,#00d2be,False,[{'fantasy_results': [{'id': 'quali_not_classi...
3,NOR,#FF9800,False,[{'fantasy_results': [{'id': 'quali_not_classi...
4,ALO,#006f62,False,[{'fantasy_results': [{'id': 'quali_not_classi...
5,SAI,#dc0000,False,[{'fantasy_results': [{'id': 'quali_not_classi...
6,RUS,#00d2be,False,[{'fantasy_results': [{'id': 'quali_not_classi...
7,LEC,#dc0000,False,[{'fantasy_results': [{'id': 'quali_not_classi...
8,PIA,#FF9800,False,[{'fantasy_results': [{'id': 'quali_not_classi...
9,STR,#006f62,False,[{'fantasy_results': [{'id': 'quali_not_classi...


In [4]:
constructors

,abbreviation,color,constructor,race_results
0,RED,#0600ef,True,[{'fantasy_results': [{'id': 'quali_not_classi...
1,MER,#00d2be,True,[{'fantasy_results': [{'id': 'quali_not_classi...
2,FER,#dc0000,True,[{'fantasy_results': [{'id': 'quali_not_classi...
3,MCL,#FF9800,True,[{'fantasy_results': [{'id': 'quali_not_classi...
4,AST,#006f62,True,[{'fantasy_results': [{'id': 'quali_not_classi...
5,ALP,#0090ff,True,[{'fantasy_results': [{'id': 'quali_not_classi...
6,ALT,#2b4562,True,[{'fantasy_results': [{'id': 'quali_not_classi...
7,ALF,#900000,True,[{'fantasy_results': [{'id': 'quali_not_classi...
8,HAA,#ffffff,True,[{'fantasy_results': [{'id': 'quali_not_classi...
9,WIL,#005aff,True,[{'fantasy_results': [{'id': 'quali_not_classi...


In [4]:
# Drop unnecessary columns
drivers.drop(columns=['color', 'constructor'], inplace=True)


def extract_values(string):
    # Convert the string representation of the list of dictionaries to an actual list
    lst = ast.literal_eval(string)

    # Extracting the required values
    weekend_points = lst[0]['results_per_race_list']
    price_at_lock = lst[1]['results_per_race_list']

    return weekend_points, price_at_lock


# Function to extract the values from the nested lists and create new columns
def extract_columns(row, column):
    return pd.Series(row[column])


# Apply the extract_values function to 'race_results' column to get two separate columns
drivers[['weekend_points', 'price_at_lock']] = pd.DataFrame(drivers['race_results'].apply(extract_values).tolist())

# Create new columns for each value within 'weekend_points' and 'price_at_lock'
weekend_points_df = pd.DataFrame(drivers['weekend_points'].tolist(), columns=[f'weekend_point_{i}' for i in range(1, 23)])
price_at_lock_df = pd.DataFrame(drivers['price_at_lock'].tolist(), columns=[f'price_at_lock_{i}' for i in range(1, 23)])

# Concatenate the new columns with the existing DataFrame
drivers = pd.concat([drivers, weekend_points_df, price_at_lock_df], axis=1)

# Drop the original 'weekend_points' and 'price_at_lock' columns
drivers.drop(columns=['race_results', 'weekend_points', 'price_at_lock'], inplace=True)

# Reorder the DataFrame based on the drivers_order list
drivers_order = ['VER', 'LEC', 'PER', 'RUS', 'SAI', 'HAM', 'NOR', 'OCO', 'ALO', 'BOT', 'MAG', 'GAS', 'STR', 'ZHO', 'TSU', 'ALB', 'DEV', 'HUL', 'PIA', 'SAR']
drivers['abbreviation'] = pd.Categorical(drivers['abbreviation'], categories=drivers_order, ordered=True)
drivers.sort_values(by='abbreviation', inplace=True)

# Creating a copy of the DataFrame to sort
drivers_to_sort = drivers.copy()

# Dropping the 'abbreviation' column as it is not required for sorting
drivers_to_sort.drop(columns=['abbreviation'], inplace=True)

# Selecting only the columns with the values to be sorted
drivers_to_sort = drivers_to_sort.iloc[:,:22]

# Sorting each data_to_sort individually
sorted_drivers = drivers_to_sort.apply(lambda x: x.sort_values(ascending=False).values)

# Reversing the sorted values to have highest at the top
reversed_drivers = sorted_drivers.apply(lambda x: x[::-1])

# Applying sorted values back to the DataFrame
for col in sorted_drivers.columns:
    sorted_drivers[col] = reversed_drivers[col]

# Calculating the mean per row
row_means = sorted_drivers.mean(axis=1)

# Creating a new column 'row_mean' to store the mean per row
sorted_drivers['mean'] = row_means.astype(int)

# Concatenate the sorted DataFrame with the original DataFrame
drivers = pd.concat([drivers, sorted_drivers['mean']], axis=1)

# Get the column names to be renamed
columns_to_rename = [col for col in drivers.columns if 'weekend_point_' in col]

# Create a dictionary to map the old column names to the new names
rename_dict = {col: f"weekend_point_{int(col.split('_')[-1])+1}" for col in columns_to_rename}

# Rename the columns
drivers.rename(columns=rename_dict, inplace=True)
drivers.rename(columns={'mean': 'weekend_point_1'}, inplace=True)

# Get the column names
columns = drivers.columns.tolist()

# Reorder the columns
new_columns = columns[:-1]  # Exclude the last column
new_columns.insert(1, columns[-1])  # Insert the last column as the second one

# Create a new DataFrame with the reordered columns
drivers = drivers[new_columns]

# Drop the unneeded column
drivers.drop(columns=['weekend_point_23'], inplace=True)

drivers_dict = {}
for col in range(1, 23):
    race_data = {}
    for idx, row in drivers.iterrows():
        driver = row['abbreviation']
        weekend_points = row[f'weekend_point_{col}']
        price_at_lock = row[f'price_at_lock_{col}']
        race_data[driver] = [price_at_lock, weekend_points]
    drivers_dict[col] = race_data

In [5]:
# Drop unnecessary columns
constructors.drop(columns=['color', 'constructor'], inplace=True)


def extract_values(string):
    # Convert the string representation of the list of dictionaries to an actual list
    lst = ast.literal_eval(string)

    # Extracting the required values
    weekend_points = lst[0]['results_per_race_list']
    price_at_lock = lst[1]['results_per_race_list']

    return weekend_points, price_at_lock


# Function to extract the values from the nested lists and create new columns
def extract_columns(row, column):
    return pd.Series(row[column])


# Apply the extract_values function to 'race_results' column to get two separate columns
constructors[['weekend_points', 'price_at_lock']] = pd.DataFrame(constructors['race_results'].apply(extract_values).tolist())

# Create new columns for each value within 'weekend_points' and 'price_at_lock'
weekend_points_df = pd.DataFrame(constructors['weekend_points'].tolist(), columns=[f'weekend_point_{i}' for i in range(1, 23)])
price_at_lock_df = pd.DataFrame(constructors['price_at_lock'].tolist(), columns=[f'price_at_lock_{i}' for i in range(1, 23)])

# Concatenate the new columns with the existing DataFrame
constructors = pd.concat([constructors, weekend_points_df, price_at_lock_df], axis=1)

# Drop the original 'weekend_points' and 'price_at_lock' columns
constructors.drop(columns=['race_results', 'weekend_points', 'price_at_lock'], inplace=True)

# Reorder the DataFrame based on the drivers_order list
constructors_order = ['RED', 'FER', 'MER', 'ALP', 'MCL', 'ALF', 'AST', 'HAA', 'ALT', 'WIL']
constructors['abbreviation'] = pd.Categorical(constructors['abbreviation'], categories=constructors_order, ordered=True)
constructors.sort_values(by='abbreviation', inplace=True)

# Creating a copy of the DataFrame to sort
constructors_to_sort = constructors.copy()

# Dropping the 'abbreviation' column as it is not required for sorting
constructors_to_sort.drop(columns=['abbreviation'], inplace=True)

# Selecting only the columns with the values to be sorted
data_to_sort = constructors_to_sort.iloc[:,:22]

# Sorting each data_to_sort individually
sorted_constructors = constructors_to_sort.apply(lambda x: x.sort_values(ascending=False).values)

# Reversing the sorted values to have highest at the top
reversed_constructors = sorted_constructors.apply(lambda x: x[::-1])

# Applying sorted values back to the DataFrame
for col in sorted_constructors.columns:
    sorted_constructors[col] = reversed_constructors[col]

# Calculating the mean per row
row_means = sorted_constructors.mean(axis=1)

# Creating a new column 'row_mean' to store the mean per row
sorted_constructors['mean'] = row_means.astype(int)

# Concatenate the sorted DataFrame with the original DataFrame
constructors = pd.concat([constructors, sorted_constructors['mean']], axis=1)

# Get the column names to be renamed
columns_to_rename = [col for col in constructors.columns if 'weekend_point_' in col]

# Create a dictionary to map the old column names to the new names
rename_dict = {col: f"weekend_point_{int(col.split('_')[-1])+1}" for col in columns_to_rename}

# Rename the columns
constructors.rename(columns=rename_dict, inplace=True)
constructors.rename(columns={'mean': 'weekend_point_1'}, inplace=True)

# Get the column names
columns = constructors.columns.tolist()

# Reorder the columns
new_columns = columns[:-1]  # Exclude the last column
new_columns.insert(1, columns[-1])  # Insert the last column as the second one

# Create a new DataFrame with the reordered columns
constructors = constructors[new_columns]

# Drop the unneeded column
constructors.drop(columns=['weekend_point_23'], inplace=True)

constructors_dict = {}
for col in range(1, 23):
    race_data = {}
    for idx, row in constructors.iterrows():
        constructor = row['abbreviation']
        weekend_points = row[f'weekend_point_{col}']
        price_at_lock = row[f'price_at_lock_{col}']
        race_data[constructor] = [price_at_lock, weekend_points]
    constructors_dict[col] = race_data

In [7]:
print(drivers_dict)
print(constructors_dict)

{1: {'VER': [26.9, 50], 'LEC': [21.2, 38], 'PER': [18.0, 31], 'RUS': [18.6, 27], 'SAI': [17.2, 23], 'HAM': [23.7, 21], 'NOR': [11.2, 18], 'OCO': [9.4, 16], 'ALO': [8.3, 13], 'BOT': [7.8, 12], 'MAG': [6.7, 10], 'GAS': [8.1, 8], 'STR': [7.5, 6], 'ZHO': [4.9, 4], 'TSU': [4.8, 3], 'ALB': [5.5, 0], 'DEV': [5.0, -1], 'HUL': [4.3, -5], 'PIA': [7.0, -11], 'SAR': [4.0, -15]}, 2: {'VER': [27.0, 35], 'LEC': [21.1, -11], 'PER': [18.1, 28], 'RUS': [18.6, 16], 'SAI': [17.2, 19], 'HAM': [23.7, 19], 'NOR': [11.1, -1], 'OCO': [9.3, -17], 'ALO': [8.5, 39], 'BOT': [7.8, 13], 'MAG': [6.7, 8], 'GAS': [8.2, 20], 'STR': [7.6, 17], 'ZHO': [4.9, 15], 'TSU': [4.8, 8], 'ALB': [5.5, 13], 'DEV': [5.0, 8], 'HUL': [4.2, -1], 'PIA': [6.9, -16], 'SAR': [4.0, 11]}, 3: {'VER': [27.1, 61], 'LEC': [21.2, 27], 'PER': [18.2, 36], 'RUS': [18.6, 18], 'SAI': [17.2, 12], 'HAM': [23.7, 16], 'NOR': [11.1, 7], 'OCO': [9.3, 7], 'ALO': [8.7, 23], 'BOT': [7.7, -2], 'MAG': [6.7, 9], 'GAS': [8.2, 6], 'STR': [7.5, -14], 'ZHO': [4.9, 6],

In [85]:
def create_combinations(data, i, defined_keys=None):
    available_keys = data[-1].keys()
    remaining_keys = available_keys - (defined_keys or set())
    defined_keys_list = list(defined_keys) if defined_keys else []
    
    data_combinations = combinations(remaining_keys, i - len(defined_keys_list))
    data_combinations = [defined_keys_list + list(data_combination) for data_combination in data_combinations]

    return data_combinations


def optimize_team(current_round, num_previous_race=1, max_capacity=100, drivers_defined=None, constructors_defined=None):
    # Initialize variables
    best_value = 0
    best_team = None
    
    # Collect data from previous races
    drivers_data = [drivers_dict[i] for i in range(current_round - num_previous_race + 1, current_round + 1)]
    constructors_data = [constructors_dict[i] for i in range(current_round - num_previous_race + 1, current_round + 1)]

    # Generate combinations of 5 drivers
    driver_combinations = create_combinations(drivers_data, 5, drivers_defined)

    # Iterate through all combinations
    for drivers_combo in driver_combinations:
        # Generate combinations of 2 constructors for each driver combination iteration
        constructor_combinations = create_combinations(constructors_data, 2, constructors_defined)

        for constructors_combo in constructor_combinations:
            # Calculate total weight for the combination
            total_weight = sum(drivers_data[-1][driver][0] for driver in drivers_combo) + sum(constructors_data[-1][constructor][0] for constructor in constructors_combo)

            # Check if within capacity limit
            if total_weight > max_capacity:
                continue
            
            # Initialize total value
            total_value = 0

            for i in range(num_previous_race):
                # Calculate mean value for each driver and constructor
                driver_value_mean = sum(drivers_data[i][driver][1] for driver in drivers_combo) / num_previous_race
                constructor_value_mean = sum(constructors_data[i][constructor][1] for constructor in constructors_combo) / num_previous_race

                # Calculate total value for the combination
                total_value += driver_value_mean + constructor_value_mean

            # Update best team
            if total_value > best_value:
                best_value = total_value
                best_team = (list(drivers_combo), list(constructors_combo))
    
    return best_value, best_team


# Usage
best_value, best_team = optimize_team(drivers_dict, constructors_dict, 4, 2, 100, drivers_defined=['VER', 'PER'])
print("Best value:", best_value)
print("Best team (drivers, constructors):", best_team)

Best value: 267.0
Best team (drivers, constructors): (['VER', 'PER', 'ALO', 'TSU', 'ZHO'], ['AST', 'RED'])


In [6]:
# save drivers and constructors dicts
import pickle

with open('F1_Fantasy\\Fantasy2023\\drivers_dict.pkl', 'wb') as f:
    pickle.dump(drivers_dict, f)
    
with open('F1_Fantasy\\Fantasy2023\\constructors_dict.pkl', 'wb') as f:
    pickle.dump(constructors_dict, f)